In [1]:
# Import libaries
import pandas as pd
import requests
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

In [2]:
headers = {'User-agent': 'SB 2.0'}

In [3]:
shows = pd.read_csv('./data/k_titles.csv')
shows.head()

,title,url,container,rating,# ratings
0,Thirty But Seventeen,https://www.viki.com/tv/36109c-thirty-but-seve...,36109c,9.58,7368
1,Fates and Furies,https://www.viki.com/tv/36240c-fates-and-furies,36240c,9.14,1401
2,The Last Empress,https://www.viki.com/tv/36241c-the-last-empress,36241c,9.45,3858
3,Encounter,https://www.viki.com/tv/36239c-encounter,36239c,9.59,5436
4,My Strange Hero,https://www.viki.com/tv/36330c-my-strange-hero,36330c,9.56,2239


In [44]:
genre = pd.read_csv('./data/genres.csv')
genre.drop(columns=['Unnamed: 0'], inplace=True)

In [45]:
genre.head()

,id,name,type
0,1g,Action & Adventure,film and tv
1,2g,Anime,film and tv
2,1057g,Beauty,creators
3,3g,Biography,film and tv
4,1017g,C-Pop,music


### Save Genre Codes

In [ ]:
url='https://api.viki.io/v4/containers/genres.json?&app=100000a' 
res = requests.get(url, headers= headers)
soup = res.json()
genres = []
for g in soup:
    genre={}
    genre['id'] = g['id']
    genre['type'] = g['genre_type']
    genre['name'] = g['name']['en']
    genres.append(genre)

k_genres = pd.DataFrame(genres)
k_genres.to_csv('./data/genres.csv')

### Get Cast Info

In [39]:
s_cont =[]

In [58]:
cast_cont= []

for row in range(len(shows)):
    s_cast={}
    show_url = shows['url'][row] + '#modal-casts'

    # use selenium webdriver as library that acts as a 
    # headless browser to access modal window
    driver = webdriver.Chrome()
    driver.implicitly_wait(30)
    driver.get(show_url)
    
    soup_1=BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    try:
        cast = soup_1.find_all('div',{'id':'modal-casts'})
        actors = cast[0].find_all('div',{'itemprop':'actor'})

        cast_lst = []
        for actor in range(len(actors)):
            name = actors[actor].find('span', {'itemprop':'name'}).text
            cast_lst.append(name)
        s_cast['title'] = shows['title'][row]
        s_cast['cast'] = cast_lst
        cast_cont.append(s_cast)
        
    except IndexError:
        actors = soup_1.find_all('div',{'itemprop':'actor'})

        cast_lst = []
        for actor in range(1,len(actors)):
            name = actors[actor].find('span', {'itemprop':'name'}).text
            cast_lst.append(name)
        s_cast['title'] = shows['title'][row]
        s_cast['cast'] = cast_lst
        cast_cont.append(s_cast)
        
    except:
        s_cast['title'] = shows['title'][row]
        s_cast['cast'] = []        
        
    #end the Selenium browser session
    driver.quit()

s_cast = pd.DataFrame(cast_cont)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.6 x86_64)


In [59]:
pd.DataFrame(cast_cont)

,cast,title
0,"[Shin Hye Sun, Yang Se Jong, Ahn Hyo Seop, Ye ...",Thirty But Seventeen
1,"[Joo Sang Wook, Lee Min Jung, Lee Ki Woo, So Y...",Fates and Furies
2,"[Shin Sung Rok, Jang Nara, Choi Jin Hyuk, Shin...",The Last Empress
3,"[Park Bo Gum, Song Hye Kyo, Jang Seung Jo, P.O...",Encounter
4,"[Yoo Seung Ho, Jo Bo Ah, Kwak Dong Yeon, Yoo S...",My Strange Hero
5,"[Park Seo Joon, Park Min Young, Lee Tae Hwan, ...",What’s Wrong With Secretary Kim
6,"[Choi Jin Hyuk, Song Ha Yoon, Lee Ho Won, Lee ...",Devilish Joy
7,"[Yoo Seung Ho, Chae Soo Bin, Uhm Ki Joon, Hwan...",I Am Not a Robot
8,"[Ji Chang Wook, Nam Ji Hyun, Choi Tae Joon, Na...",Suspicious Partner
9,"[Nam Joo Hyuk, Lee Sung Kyung, Lee Jae Yoon, K...",Weightlifting Fairy Kim Bok Joo


### Get other show info

In [5]:
# Find show ratings and # of ratings
for row in range(len(shows['container'])):
    url='https://api.viki.io/v4/containers/%s.json?app=100000a' %shows['container'][row]
    res = requests.get(url, headers= headers)
    json_pg = res.json()
    
    info ={}
    info['title'] = shows['title'][row]
    info['genre'] = json_pg['genres']
    info['s_rating'] = json_pg['rating']
    info['# episodes'] = json_pg['episodes']['count']
    info['network'] = json_pg['distributors'][0]['name']
    info['rating']= json_pg['review_stats']['average_rating']
    s_cont.append(info)
    time.sleep(1)

s_contents = pd.DataFrame(s_cont)

In [40]:
s_contents.head()

,# episodes,genre,network,rating,s_rating,title,cast
0,32,"[23g, 9g, 18g, 6g, 7g]",SBS,9.58,PG-13,Thirty But Seventeen,NaN
1,36,"[18g, 1041g, 23g]",SBS,8.73,PG-13,Fates and Furies,NaN
2,42,"[23g, 9g, 18g]",SBS,9.45,PG-13,The Last Empress,NaN
3,16,"[18g, 23g]",tvN,9.56,PG-13,Encounter,NaN
4,32,"[18g, 1041g, 9g, 23g]",SBS,9.55,PG-13,My Strange Hero,NaN


In [41]:
s_contents.to_csv('./data/s_contents.csv', index = False)

In [ ]:
s_contents['s_rating'].value_counts()

In [ ]:
shows['s_title']=shows['title'].map(lambda x: x.replace(' ','_'))

In [ ]:
shows['s_title'][23]

In [ ]:
s_info =[]

In [ ]:
url3='https://wiki.d-addicts.com/You_Who_Came_From_the_Stars' #%shows['s_title'][21] 
res = requests.get(url3, headers = {'User-agent': 'SB 2.0'})
soup = BeautifulSoup(res.content, 'lxml')

info = soup.find('div',{'class':'mw-parser-output'})
table = info.find_all('ul')

In [ ]:
table

In [ ]:
content = {}
content['title'] = shows['title'][22]
content['genre'] = re.findall('Genre:\s(.+)',table[2].text)
content['network'] = re.findall('Broadcast network:\s(.+)',table[2].text)[0]
content['cast'] = re.findall('([A-z].*?)\sas\s', table[3].text)
content['director'] = re.findall('Director:\s(.+)', table[14].text)[0]
content['screenwriter'] = re.findall('Screenwriter:\s(.+)', table[14].text)[0]
table[14]


In [ ]:
content

In [ ]:
s_info.append(content)

In [ ]:
pd.DataFrame(s_info)